# Algarve piezometric values for all aquifer locations
* Notebook adapted from great works of Luis Costa from APA
* single station data from `aww_output/single_station_data` are linked `algarve_stations.csv` table via `site_id`

TODO: convert this to two scripts
1. runs only once upon a time to confirm station updates etc
2. runs every month to get new data
    * add asserts
    * update existing data, rather then download all


In [43]:
import pandas as pd
import requests
import io

from tqdm import tqdm
import json

## Setup

In [74]:
# for requests
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/118.0",
}

# Function to get the representative piezometer name, from the dict
def get_key(d: dict, val: str):
    for key, value in d.items(): # give name of piezometer, get code
         if val == value:
             return key
 
    return "key doesn't exist"

## 1 - Creating a dictionary of station IDs and code

Query from website with station info (including Id and code) have been stored at "station_list.csv"

Stations list: https://snirh.apambiente.pt/snirh/_dadosbase/site/xml/xml_listaestacoes.php

Info to retreive
marker site -> hiperlink code
estacao -> Station ID
lat -> Latitude
lng -> Longitude

## 1.1 - reading the data and converting it to a dictionary

In [6]:
df_station = pd.read_csv('input/station_list.csv')
df_station = df_station["<markers>"].str.split('"', expand = True)

In [ ]:
#cols to drop
cols_drop = [0,2,3,4,5,6,8,10,11,12,14,16,17,18,19,20]
df_station.drop(df_station.columns[cols_drop],axis=1,inplace=True)

# Change the column names
df_station.columns =['marker site','latitude','longitude','estacao3','estacao']

# marker site is the hyperlink in the website SNIRH to reach each station

#clean the estacao3 and estacao fields
df_station['estacao3'] = df_station['estacao3'].str.replace('&amp;#9632; ','')
df_station['estacao'] = df_station['estacao'].str.replace('■ ','')
df_station.drop(['estacao3'], axis=1, inplace=True)

df_station.head()

,marker site,latitude,longitude,estacao
0,2028876,42.0474,-8.38867,3/N1
1,2028878,42.05139,-8.50448,3/N2
2,2028880,42.08108,-8.45636,3/N3
3,2004560,41.74123,-8.64391,28/1
4,2028884,41.79257,-7.36325,34/N1


In [64]:
df_station.to_csv('aww_output/station_list_clean.csv')

## 1.2 Creating a dataframe with the dictionary and all the relevant information of the observation stations 

Create a column with the corresponding "aquifer", a column with the "Região Hidrográfica", a column with the "altitude"

In [65]:
#getting the list of station and info from csv in pc (same as below, but retrieiving data from computer instead of internet)

df_local = pd.read_csv('input/rede_Piezometria.csv', encoding = "ISO-8859-1", skiprows=3, index_col=False)
df_local.head()

,CÓDIGO,NOME,DISTRITO,CONCELHO,FREGUESIA,BACIA,ALTITUDE (M),COORD_X (M),COORD_Y (M),SISTEMA AQUÍFERO,ESTADO
0,420/8,01,SETÚBAL,MONTIJO,CANHA,TEJO,44.0,152403.4,204253.4,T3 - BACIA DO TEJO-SADO / MARGEM ESQUERDA,NaN
1,420/9,02,SETÚBAL,MONTIJO,CANHA,TEJO,44.0,152404.1,204254.6,T3 - BACIA DO TEJO-SADO / MARGEM ESQUERDA,NaN
2,405/67,1 AH,SANTARÉM,BENAVENTE,SAMORA CORREIA,TEJO,14.0,137569.9,218952.4,T3 - BACIA DO TEJO-SADO / MARGEM ESQUERDA,NaN
3,405/68,1 AH,SANTARÉM,BENAVENTE,SAMORA CORREIA,TEJO,14.0,137570.0,208900.0,T3 - BACIA DO TEJO-SADO / MARGEM ESQUERDA,NaN
4,405/69,1 AH,SANTARÉM,BENAVENTE,SAMORA CORREIA,TEJO,14.0,137571.0,208900.0,T3 - BACIA DO TEJO-SADO / MARGEM ESQUERDA,NaN


In [68]:
# NOTE: this retrieves an empty table, therefore below I use the df2_local

# url2 = 'https://snirh.apambiente.pt/snirh/_dadosbase/site/paraCSV/lista_csv.php?obj_janela=INFO_ESTACOES&s_cover=100290946&tp_lista=&completa=1&formato=csv'

# r = requests.get(url2, headers=headers, allow_redirects=True).content # getting the file
# df_web = pd.read_csv(io.StringIO(r.decode("ISO-8859-1")), skiprows=3, index_col=False)

# df_web.head()

In [ ]:
# merging the df with the hyperlink code to the df with the stations caracteristics

df_stations = pd.merge(df_local, df_station,
                       left_on='CÓDIGO', right_on='estacao',
                       how='outer', indicator=True) #merge the df with hyperlink - station relation, with the df with info on the stations
df_stations = df_stations.iloc[:-1 , :] #drop the last row which is uneeded info
df_stations = df_stations[df_stations['_merge'] != 'left_only'] # to remove obs point 607/884 that was added to the list, but was abandanoed, and it does not show up in the XML list

df_stations

,CÓDIGO,NOME,DISTRITO,CONCELHO,FREGUESIA,BACIA,ALTITUDE (M),COORD_X (M),COORD_Y (M),SISTEMA AQUÍFERO,ESTADO,marker site,latitude,longitude,estacao,_merge
0,104/N2,A11P,BRAGANÇA,VILA FLOR,VILARINHO DAS AZENHAS,DOURO,462.00,279820.0,488342.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,2028852,41.36038,-7.17921,104/N2,both
1,113/N1,AD9,PORTO,AMARANTE,SALVADOR DO MONTE,DOURO,285.00,203095.0,473597.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,2028828,41.23156,-8.0962,113/N1,both
2,114/N1,AD5,VILA REAL,VILA REAL,CAMPEÃ,DOURO,702.00,223188.0,479509.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,2028894,41.28446,-7.85632,114/N1,both
3,118/N1,AD4,BRAGANÇA,TORRE DE MONCORVO,ADEGANHA,DOURO,124.00,286908.0,473920.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,2028890,41.22981,-7.09657,118/N1,both
4,137/N1,AF31,VISEU,TAROUCA,TAROUCA,DOURO,507.00,228818.0,451870.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,2028888,41.03542,-7.79042,137/N1,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985,69/N1,AF20,BRAGA,BARCELOS,MIDÕES,CÁVADO/RIBEIRAS COSTEIRAS,86.00,162493.0,503742.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,2028904,41.50211,-8.58233,69/N1,both
986,74/N1,AF17,VILA REAL,VILA POUCA DE AGUIAR,GOUVÃES DA SERRA,DOURO,868.00,235233.0,501889.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,2028892,41.48553,-7.71125,74/N1,both
987,82/16,NaN,PORTO,PÓVOA DE VARZIM,AGUÇADOURA,CÁVADO/RIBEIRAS COSTEIRAS,9.75,146290.0,497150.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,17140,41.44184,-8.7758,82/16,both
988,96/25,NaN,PORTO,PÓVOA DE VARZIM,ARGIVAI,AVE/LEÇA,15.24,149800.0,489120.0,A0 - MACIÇO ANTIGO INDIFERENCIADO,NaN,17149,41.36977,-8.73313,96/25,both


## 2. Getting the data

### 2.1. Setting the filter for the zones in PT. 
* In this case, we want  only Algarve
* select piezometers based on "BACIA"
* save DF to csv: `algarve_stations.csv`

In [58]:
#Define a df with all the river basins covering Algarve, so we can select all obs pointsfrom it after.
Bacias = df_local['BACIA'].unique() #view all the available "BACIAS"
Bacias


array(['TEJO', 'RIBEIRAS DO ALGARVE', 'GUADIANA', 'SADO', 'DOURO',
       'LIMA/NEIVA', 'MINHO/ÂNCORA', 'VOUGA/RIBEIRAS COSTEIRAS',
       'LIS/RIBEIRAS COSTEIRAS', 'RIBEIRAS DO OESTE', 'MONDEGO',
       'RIBEIRAS DO ALENTEJO', 'CÁVADO/RIBEIRAS COSTEIRAS', 'ARADE',
       'AVE/LEÇA', nan], dtype=object)

"BACIAS" for Algarve: 'RIBEIRAS DO ALGARVE','GUADIANA','ARADE'

TODO: site_id should be made into index I think

In [62]:
#Selecting all sites of the Algarve, considering the ones selected in the Algarve "BACIAS"
sites_Algarve = df_stations.loc[df_stations['BACIA'].isin(['RIBEIRAS DO ALGARVE','GUADIANA','ARADE'])]

# remove duplicate columns or columns with no value
sites_Algarve = sites_Algarve.drop(["_merge", "CÓDIGO"], axis=1)
sites_Algarve = sites_Algarve.rename(columns={"estacao": "site_id"})

# save and show
sites_Algarve.to_csv("aww_output/algarve_stations.csv", header = True)
sites_Algarve.head()

,NOME,DISTRITO,CONCELHO,FREGUESIA,BACIA,ALTITUDE (M),COORD_X (M),COORD_Y (M),SISTEMA AQUÍFERO,ESTADO,marker site,latitude,longitude,site_id
391,Salvador,PORTALEGRE,CAMPO MAIOR,SÃO JOÃO BAPTISTA,GUADIANA,190.00,299975.0,231625.0,A11 - ELVAS - CAMPO MAIOR,NaN,34441,39.04669,-6.97828,387/4
392,NaN,PORTALEGRE,CAMPO MAIOR,-,GUADIANA,233.00,297866.0,231805.0,A11 - ELVAS - CAMPO MAIOR,NaN,2077786,39.04855,-7.00261,387/8
419,Pena Clara,PORTALEGRE,ELVAS,SÃO VICENTE E VENTOSA,GUADIANA,380.00,275550.0,220125.0,A5 - ELVAS - VILA BOIM,NaN,34459,38.94556,-7.26165,399/12
420,NaN,PORTALEGRE,ELVAS,SÃO VICENTE E VENTOSA,GUADIANA,382.35,275702.0,220199.0,A5 - ELVAS - VILA BOIM,NaN,17288,38.94622,-7.25989,399/6
424,NaN,PORTALEGRE,CAMPO MAIOR,NOSSA SENHORA DA EXPECTAÇÃO,GUADIANA,200.00,292075.0,221525.0,A11 - ELVAS - CAMPO MAIOR,NaN,34563,38.95659,-7.07088,400/7


### 2.2. Getting the data with a For Loop to create an individual csv file for each Obs Point

In [ ]:
# dictionary with obs points in Sites_Algarve 
station_Algarve_dict = dict(zip(sites_Algarve['marker site'], sites_Algarve['estacao']))

# dictionary with obs points code and belonging aquifer
station_aquif_Algarve_dict = dict(zip(sites_Algarve['marker site'], sites_Algarve['SISTEMA AQUÍFERO']))

In [46]:
# Define filter Parameters in this case hydrological years 1990 until 2025
iDate = '01/10/1990' #Initial date
eDate = '28/05/2025' #End date
par = '2277' #depth to groundwater level
print(f"number of stations to be saved (1 csv per station): {len(list(station_aquif_Algarve_dict.keys()))}")

number of stations to be saved (1 csv per station): 402


In [44]:
for key in tqdm(list(station_aquif_Algarve_dict.keys())): #for all obs points within the selected aquifers
    PiezometerName = station_Algarve_dict[key]
    PiezometerCode = get_key(station_Algarve_dict, PiezometerName)

    url = 'https://snirh.apambiente.pt/snirh/_dadosbase/site/paraCSV/dados_csv.php?sites='+key+'&pars='+par+'&tmin=' + iDate + '&tmax=' + eDate + '&formato=csv'
    r = requests.get(url, headers=headers, allow_redirects=True).content # getting the file
    df = pd.read_csv(io.StringIO(r.decode("ISO-8859-1")), skiprows=3, dayfirst=True,
                     parse_dates = [0], index_col = [0], usecols = [0,1], header = 0, skipfooter = 1,
                     names = ['Date', PiezometerName], engine = 'python') 
    
    filepath = './aww_output/single_station_data/' + PiezometerCode + '.csv'
    df.to_csv(filepath, header = True)

100%|██████████| 402/402 [03:16<00:00,  2.05it/s]


## 3. Example code of how to query the data from SNIRH

In [72]:
# iDate = '01/01/1990' #Initial date
# eDate = '31/12/1995' #End date
# sites = '107724,107731,107753,2047288,71842,2046000,2077730,107825,106626,109128,107077,107073,107074,107098,107099,107066,71844,107119,107519,106628,107259,107557,107260,107272,107233,107560,107219,107252,107251,107240,107550,107340' #hiperlink code for the obs points
# pars = '100290981' #piezometric level 

# # url def 
# url = 'https://snirh.apambiente.pt/snirh/_dadosbase/site/paraCSV/dados_csv.php?sites=' + sites + '&pars=' + pars + '&tmin=' + iDate + '&tmax=' + eDate + '&formato=csv'
# # in url, 
# #       sites=... selects the observation points based on their hyperlink code; 
# #       par = selection of the parameter we want to download (piezometria ou profunidade ao nível) based on the specific hyperlink code
# #       tmin and tmax = initial and end date


# r = requests.get(url, headers=headers, allow_redirects=True).content # getting the file
# df_test = pd.read_csv(io.StringIO(r.decode("ISO-8859-1")), skiprows=3)
# df_test.head()

In [73]:
# df_test.drop(df_test.tail(1).index, inplace = True) #dropping last row with unwanted text
# df_test = df_test.rename(columns={'Unnamed: 0': 'Date'}) #renaming dateindex column
# df_test = df_test.drop(['Unnamed: 65'],axis=1) 
# df_test = df_test.set_index('Date') #transforming into dateindex
# df_test = df_test.drop(df_test.filter(regex='FLAG').columns, axis=1) #dropping blank columns - Attention this is should only be done after elimination of "<" or ">" values of the corrseponding observation point. 
# df_test.columns = ['606/1026','606/1033','606/1057','606/1118','606/1122','606/1461','606/1639','606/167',
#               '606/647','607/498','610/167','610/179','610/180',
#               '610/182','610/183','610/186','610/213','610/6',
#               '610/65','611/115','611/155','611/182','611/200',
#               '611/202','611/209','611/217','611/230','611/233',
#               '611/234','611/236','611/237','611/91']

# df_test.head()